# Capstone Peoject- The Battle of Neighbors 

## Introduction 

My client is planning to open a new Pizza Restaturant in New York. However, my client is uncertain about the location. He wants to compare several places first before making the final decison. There are several areas in New York will be used as the comparision. The client wants to know that compare to Brooklyn and Queens, **is Manhattan the best place to open a new Pizza Restaurant?** In addition, the busisness owner doesn't want to open a restaurant that is close to it's competetors. Insdead, the busines owner wants to open the restaurant near bars. 

As business and Data Anlytist for my client my tasks are:

1. Find locations with the least number of pizza restaurant 
2. Among the locations we find above, select the one with the most number of bars 

## Data 

[New York Neighborhood](https://geo.nyu.edu/catalog/nyu_2451_34572)
Will use the data to analyze the Neigborhood of Manhatton, Queens, and Brooklyn.

### Download Data and Explore Dataset 

In [1]:
import pandas as pd
import numpy as np
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim #covert address to lat and lot values 
import requests #handel requests
import matplotlib.cm as cm
import matplotlib.colors as colors 
from sklearn.cluster import KMeans

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

In [154]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Done')

Done


In [155]:
import json
from pandas.io.json import json_normalize
print('Done')

Done


In [156]:
with open ('newyork_data.json') as ny_data:
    newyork_data = json.load(ny_data)

In [160]:
#newyork_data

In [161]:
nb_data=newyork_data['features']

In [162]:
nb_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [163]:
column_names =['Borough', 'Neighborhood', 'Latitude', 'Longitude']
neighbors =pd.DataFrame(columns =column_names)
neighbors


,Borough,Neighborhood,Latitude,Longitude


In [164]:
for data in nb_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighbors = neighbors.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [165]:
neighbors.head(20)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


#### Manhattan Data

In [166]:
man = neighbors[neighbors['Borough'] == 'Manhattan'].reset_index(drop=True)
man.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [167]:
address = 'Manhattan'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [168]:
CLIENT_ID = '32ZO5WWBYFCG3GBPVF0CBCG5P2ILDUJ1BNFOSILFGX51W2KU' # your Foursquare ID
CLIENT_SECRET = 'DLI1WNKOGLEXRKXY322Z4GPSQIVKFUMD2EH3DPLNYUJIH4XX' # your Foursquare Secret
VERSION = '20200721'

In [169]:
man.loc[0, 'Neighborhood']

'Marble Hill'

#### Queens

In [172]:
q = neighbors[neighbors['Borough'] == 'Queens'].reset_index(drop=True)
q.head(5)

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


In [176]:
address = 'Queens'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Queens are 40.7498243, -73.7976337.


In [174]:
q.loc[0, 'Neighborhood']

'Astoria'

In [180]:
b = neighbors[neighbors['Borough'] == 'Brooklyn'].reset_index(drop=True)
b.head(5)

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [177]:
address = 'Brooklyn'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brookln are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brookln are 40.6501038, -73.9495823.


In [182]:
b.loc[10, 'Neighborhood']

'East Flatbush'